In [58]:
# This notebook allows to read the NGFS data from csv files provided by NGFS and create the required data files
import numpy as np
import pandas as pd
import pickle
def getNGFSvar(variable,model,years,df,fillNaN=False):
    scenarios = ['Below 2°C', 'Current Policies', 'Delayed transition', 
                 'Divergent Net Zero', 'Nationally Determined Contributions (NDCs)', 'Net Zero 2050']
    dfm = df[df["Model"]==model]
    res = np.zeros((len(scenarios),len(years)))
    dfv = dfm[dfm["Variable"]==variable]
    for s in range(len(scenarios)):
        dfs = dfv[dfv["Scenario"]==scenarios[s]]
        for y in range(len(years)):
            res[s,y] = float(dfs[str(years[y])])
        if(fillNaN):
            res[s,:] = np.interp(years,years[~np.isnan(res[s,:])],res[s,~np.isnan(res[s,:])])
    return res

In [14]:
# Read the NGFS data files 
# This may take a long time
# the files are downloaded from https://data.ene.iiasa.ac.at/ngfs/#/downloads
df_downscaled = pd.read_excel("1665743477883-V3.4-NGFS-Phase-3/Downscaled_data.xlsx")
df_deu = df_downscaled[df_downscaled["Region"]=="DEU"]

df_iam = pd.read_excel("1665743477883-V3.4-NGFS-Phase-3/IAM_data.xlsx")
df_europe = df_iam[df_iam["Region"]=="REMIND-MAgPIE 3.0-4.4|EU 28"]



In [101]:
# Variables for the optimal exit problem
years_exit = np.arange(2020,2055,5)
P_E = getNGFSvar("Price|Secondary Energy|Electricity","Downscaling [REMIND-MAgPIE 3.0-4.4]",years_exit,df_deu)*1.19*0.86*3.6
P_C = getNGFSvar("Price|Primary Energy|Coal","Downscaling [REMIND-MAgPIE 3.0-4.4]",years_exit,df_deu)*1.19*0.86*3.6
P_CO2 = getNGFSvar("Price|Carbon","Downscaling [REMIND-MAgPIE 3.0-4.4]",years_exit,df_deu)*1.19*0.86
R_U = getNGFSvar("Secondary Energy|Electricity|Coal|w/o CCS","REMIND-MAgPIE 3.0-4.4",years_exit,df_europe)/getNGFSvar("Capacity|Electricity|Coal|w/o CCS","REMIND-MAgPIE 3.0-4.4",years_exit,df_europe)*1000000/365.25/24/3.600
emissions = getNGFSvar("Emissions|CO2|Energy","Downscaling [REMIND-MAgPIE 3.0-4.4]",years_exit,df_deu)*1e6
CC = getNGFSvar("Capital Cost|Electricity|Coal|w/o CCS","REMIND-MAgPIE 3.0-4.4",years_exit,df_europe)[0,:]*1.19*0.86*1e3
R_U[R_U>1] = 1
# Prices are in USD2010. Need to convert them to EUR 2020
# 1 USD 2010 = 1.19 USD 2020 (see
# https://www.statista.com/statistics/1032048/value-us-dollar-since-1640/)
# FX rate USD to EUR in 2020: 0.86
# The factor 3.6 is to convert price per GJ into price per MWh

pickle.dump({"P_E":P_E,"P_C":P_C,"P_CO2":P_CO2,"R_U":R_U,"emissions":emissions,"CC":CC}, open( "ngfs_variables.p", "wb" ) )




In [100]:
# Variables for the optimal entry problem
years_entry = np.arange(2020,2101,5)
P_B = getNGFSvar("Price|Primary Energy|Biomass","REMIND-MAgPIE 3.0-4.4",years_entry,df_europe,fillNaN=True)*1.19*0.86*3.6
P_B[:,:len(years_exit)] = getNGFSvar("Price|Primary Energy|Biomass","Downscaling [REMIND-MAgPIE 3.0-4.4]",years_exit,df_deu)*1.19*0.86*3.6
P_E = getNGFSvar("Price|Secondary Energy|Electricity","REMIND-MAgPIE 3.0-4.4",years_entry,df_europe,fillNaN=True)*1.19*0.86*3.6
P_E[:,:len(years_exit)] = getNGFSvar("Price|Secondary Energy|Electricity","Downscaling [REMIND-MAgPIE 3.0-4.4]",years_exit,df_deu)*1.19*0.86*3.6
P_CO2 = getNGFSvar("Price|Carbon","REMIND-MAgPIE 3.0-4.4",years_entry,df_europe,fillNaN=True)*1.19*0.86
P_CO2[:,:len(years_exit)] = getNGFSvar("Price|Carbon","Downscaling [REMIND-MAgPIE 3.0-4.4]",years_exit,df_deu)*1.19*0.86
P_C = getNGFSvar("Price|Primary Energy|Coal","REMIND-MAgPIE 3.0-4.4",years_entry,df_europe,fillNaN=True)*1.19*0.86*3.6
P_C[:,:len(years_exit)] = getNGFSvar("Price|Primary Energy|Coal","Downscaling [REMIND-MAgPIE 3.0-4.4]",years_exit,df_deu)*1.19*0.86*3.6

R_U = getNGFSvar("Secondary Energy|Electricity|Coal|w/o CCS","REMIND-MAgPIE 3.0-4.4",years_entry,df_europe,fillNaN=True)/getNGFSvar("Capacity|Electricity|Coal|w/o CCS","REMIND-MAgPIE 3.0-4.4",years_entry,df_europe,fillNaN=True)*1000000/365.25/24/3.600

emissions = getNGFSvar("Emissions|CO2|Energy","REMIND-MAgPIE 3.0-4.4",years_entry,df_europe,fillNaN=True)*1e6
emissions[:,:len(years_exit)] = getNGFSvar("Emissions|CO2|Energy","Downscaling [REMIND-MAgPIE 3.0-4.4]",years_exit,df_deu)*1e6


R_U_B = (getNGFSvar("Secondary Energy|Electricity|Biomass|w/o CCS","REMIND-MAgPIE 3.0-4.4",years_entry,df_europe,fillNaN=True)/
         getNGFSvar("Capacity|Electricity|Biomass|w/o CCS","REMIND-MAgPIE 3.0-4.4",years_entry,df_europe,fillNaN=True))*1000000/365.25/24/3.600

CC_B = getNGFSvar("Capital Cost|Electricity|Biomass|w/o CCS","REMIND-MAgPIE 3.0-4.4",years_entry,df_europe,fillNaN=True)[0,:]*1.19*0.86*1e3

CC_C = getNGFSvar("Capital Cost|Electricity|Coal|w/ CCS","REMIND-MAgPIE 3.0-4.4",years_entry,df_europe,fillNaN=True)[0,:]*1.19*0.86*1e3



R_U_B[R_U_B>1] = 1
R_U[R_U>1] = 1


pickle.dump({"P_B":P_B,"P_E":P_E,"P_C": P_C,"P_CO2":P_CO2,"R_U":R_U,"R_U_B": R_U_B,"CC_B":CC_B,
             "CC_C":CC_C,"emissions": emissions}, open( "ngfs_variables_entry.p", "wb" ) )
